### Imports and Convenience Functions

In [8]:
import warnings
import numpy as np
import phe as paillier
from sonar.contracts import ModelRepository,Model
from sonar.ipfs import IPFS
from syft.he.paillier.keys import KeyPair
from syft.nn.linear import LinearClassifier
from sklearn.datasets import load_diabetes
from sonar.blockchains import connect_ethereum_infura
from ethereum.tools.keys import decode_keystore_json
import json

def get_balance(account):
    return repo.web3.fromWei(repo.web3.eth.getBalance(account),'ether')

warnings.filterwarnings('ignore')


Failed to import bitcoin. This is not a fatal error but does
mean that you will not be able to determine the address from
your wallet file.


## Setup Settings

In [10]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_diabetes()
y = diabetes.target
X = diabetes.data

validation = (X[0:5],y[0:5])
anonymous_diabetes_users = (X[6:],y[6:])

# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

andreas_repo = '0xd60e1a150b59a89a8e6e6ff2c03ffb6cb4096205'
stefan_repo = '0xf0DE3D665D777d12609D0524c89C6456A6658E5E'

wallet_json = json.loads('{"address":"c6e1d4501d3f354e89f048f096db4424168682a9","crypto":{"cipher":"aes-128-ctr","ciphertext":"739e9d49ebb3191a74817a2f007eec5717c592eb378096ca021d67a8af33e938","cipherparams":{"iv":"e283943a488e2b906c2bd68a4873a834"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"3ba0e9f38a1110e1ff257308fcbf89e27f99845b16d83b66ce6b2118f5672d55"},"mac":"66ba81180c4074d0439698a262b4596736cd568b421868dddf5fab3c99f32d73"},"id":"137dc832-4196-4fa0-b1f9-895df090071a","version":3}')
wallet = decode_keystore_json(wallet_json,'')

# ATTENTION: copy paste the correct address (NOT THE DEFAULT SEEN HERE) from truffle migrate output.
repo = ModelRepository(andreas_repo, account=my_wallet_address,ipfs=IPFS(host='https://ipfs.infura.io'), web3=connect_ethereum_infura('CV4Z8Hx9qUUzHq8PGbPP')) # blockchain hosted model repository

Connected to OpenMined ModelRepository:0xd60e1a150b59a89a8e6e6ff2c03ffb6cb4096205


In [33]:
len(repo)

12

## Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [13]:
pubkey,prikey = KeyPair().generate(n_length=1024)
diabetes_classifier = LinearClassifier(desc="DiabetesClassifier",n_inputs=10,n_labels=1)

In [14]:
initial_error = diabetes_classifier.evaluate(validation[0],validation[1])

In [15]:
diabetes_classifier.encrypt(pubkey)

Linear Model (10,1): DiabetesClassifier

In [16]:
diabetes_model = Model(owner='0xc6E1D4501D3f354E89F048F096db4424168682a9',
                       syft_obj = diabetes_classifier,
                       bounty = 0.00000000001,
                       initial_error = initial_error,
                       target_error = 10000
                      )

In [17]:
# t = repo.get_transaction(my_wallet_address)

In [32]:
from ethereum.transactions import Transaction

In [21]:
tx = Transaction(
    nonce=repo.web3.eth.getTransactionCount(my_wallet_address),
    gasprice=repo.web3.eth.gasPrice,
    startgas=100000,
    to=andreas_repo,
    value=1,
    data=b'',
)

In [22]:
import rlp

In [23]:
import json 

In [91]:
def get_transaction(self, from_addr, value=None):
    """I consistently forget the conventions for executing transactions against
    compiled contracts. This function helps that to be easier for me."""

    txn = {}
    txn["from"] = from_addr
    txn["to"] = self.contract_address

    if value is not None:
        txn["value"] = int(value)

    transact_raw = self.contract.transact(txn)
    return transact_raw
repo.get_transaction = get_transaction

deploy_tx = repo.get_transaction(repo,
            model.owner,
            value=repo.web3.toWei(model.bounty, 'ether'))

In [97]:
tx

<Transaction(44d0)>

In [26]:
signed_tx = tx.sign(wallet)

In [96]:
signed_tx

<Transaction(44d0)>

In [27]:
raw_tx = rlp.encode(tx)

In [95]:
raw_tx

b"\xf8e\x1e\x85\x17E{\xf7\x80\x83\x01\x86\xa0\x94\xd6\x0e\x1a\x15\x0bY\xa8\x9a\x8eno\xf2\xc0?\xfbl\xb4\tb\x05\x01\x80\x1b\xa0p\x08\xbfB\x18.\x14\xfcWQ%H\xb1,\xa0\xae]'0\xc8\x14Yo\x84+E\xd55Z\xf3\xe8\xee\xa0_\x12\xb0O'\xc3\xba\x89#\x81=/\xdc\x14 \x86\xe2]\xac\xf2\xd6#\x8f\x9a\xc4M\xa0\x80\xf3\xec\xffO"

In [29]:
raw_tx_hex = repo.web3.toHex(raw_tx)

In [94]:
raw_tx_hex

'0xf8651e8517457bf780830186a094d60e1a150b59a89a8e6e6ff2c03ffb6cb409620501801ba07008bf42182e14fc57512548b12ca0ae5d2730c814596f842b45d5355af3e8eea05f12b04f27c3ba8923813d2fdc142086e25dacf2d6238f9ac44da080f3ecff4f'

In [31]:
repo.web3.eth.sendRawTransaction(raw_tx_hex)

'0x44d023dbcb7d617937e7b65c09396fee0f53058e87a4fcd8cf83e3d17bb0094d'

In [93]:
andreas_repo

'0xd60e1a150b59a89a8e6e6ff2c03ffb6cb4096205'

'0xc6E1D4501D3f354E89F048F096db4424168682a9'

In [79]:
repo.contract.transact()

<function web3.contract.Contract.transact>

In [86]:
model_id = repo.submit_model(diabetes_model)

TypeError: get_transaction() missing 1 required positional argument: 'from_addr'

## Step 2: An Anonymous Patient Downloads the Model and Improves It

In [15]:
model_id = 10
model = repo[model_id]

In [16]:
repo[model_id].submit_gradient(my_wallet_address,anonymous_diabetes_users[0][0],anonymous_diabetes_users[0][0])

## Step 3: Cure Diabetes Inc. Evaluates the Gradient 

In [17]:
repo[model_id]

Desc:DiabetesClassifier
Owner:0xc6e1d4501d3f354e89f048f096db4424168682a9
Bounty:9.999999E-12
Initial Error:21802517
Best Error:None
Target Error:10000
Model ID:10
Num Grads:0

In [18]:
old_balance = get_balance(my_wallet_address)
print(old_balance)

3.709448782750000015


In [21]:
grad = model[0]

In [22]:
grad.id

2

In [25]:
model = repo[model_id]

In [26]:
new_error = model.evaluate_gradient(my_wallet_address,repo[model_id][0],prikey,pubkey,validation[0],validation[1])

In [27]:
new_error

21802481

In [30]:
new_balance = get_balance(my_wallet_address)
incentive = new_balance - old_balance
print(incentive)

-0.001333360000000000


## Step 4: Rinse and Repeat

In [31]:
model

Desc:DiabetesClassifier
Owner:0xc6e1d4501d3f354e89f048f096db4424168682a9
Bounty:9.999999E-12
Initial Error:21802517
Best Error:None
Target Error:10000
Model ID:10
Num Grads:1

In [33]:
# for i,(addr, input, target) in enumerate(anonymous_diabetics):
#     try:
        
#         model = repo[model_id]
        
#         # patient is doing this
#         model.submit_gradient(addr,input,target)
        
#         # Cure Diabetes Inc does this
#         old_balance = get_balance(addr)
#         new_error = model.evaluate_gradient(cure_diabetes_inc,model[i+1],prikey,pubkey,validation[0],validation[1],alpha=2)
#         print("new error = "+str(new_error))
#         incentive = round(get_balance(addr) - old_balance,5)
#         print("incentive = "+str(incentive))
#     except:
#         "Connection Reset"